Select nep.txt from UNEP.

In [4]:
import numpy as np

select_symbol = ['W', 'Ta', 'V', 'Cr']
n_max = np.array([6, 4])
basis_size = np.array([12, 8])
neuron = 80

comment = []
para = []
with open('unep.txt', 'r') as fid:
    lines = iter(fid.readlines())
    for line in lines:
        line = line.strip()
        input_para = line.split()[0]
        if input_para == 'nep4_zbl':
            initial_symbol = line.split()[2:]
            select_symbol = sorted(select_symbol, key=initial_symbol.index)
            comment.append(f"nep4_zbl {len(select_symbol)} " + " ".join(select_symbol))
        else:
            comment.append(line)
        if input_para == 'ANN':
            break
    for line in lines:
        para.append(float(line.split()[0]))

para = np.array(para)
N_des = n_max[0] + 1 + (n_max[1] + 1) * 5
N_ann_one = (N_des + 2) * neuron
N_c_radial = (n_max[0] + 1) * (basis_size[0] + 1) 
N_c_angular = (n_max[1] + 1) * (basis_size[1] + 1)

with open("nep.txt", "w") as fid:
    for line in comment:
        fid.write(f"{line}\n")

    offset = 0
    for symbol in initial_symbol:
        for _ in range(N_ann_one):
            if symbol in select_symbol:
                fid.write(f"{para[offset]:15.7e}\n")
            offset += 1

    fid.write(f"{para[offset]:15.7e}\n")
    offset += 1

    for m in range(N_c_radial + N_c_angular):
        for n1 in initial_symbol:
            for n2 in initial_symbol:
                if n1 in select_symbol and n2 in select_symbol:
                    fid.write(f"{para[offset]:15.7e}\n")
                offset += 1

    for n in range(N_des):
        fid.write(f"{para[offset]:15.7e}\n")
        offset += 1

Fit ZBL Parameters.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from wizard.io import read_xyz

def quadratic(r, p1, p2, p3, p4, p5, p6):
    Z1 = 74
    Z2 = 74
    a = 0.46848 / (Z1 ** 0.23 + Z2 ** 0.23) 
    A = 14.399645 * Z1 * Z2
    x = r / a
    return A / r * (p1 * np.exp(-p2 * x) + p3 * np.exp(-p4 * x) + p5 * np.exp(-p6 * x)) 

dimer = read_xyz('dimer.xyz')
r = []
y = []
p0 = [0.5, 0.5, 0.5, 0.5, 0.5, 0.55]
bounds = ([0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [1, 1, 1, 1, 1, 1])
for atoms in dimer:
    r.append(np.linalg.norm(atoms[0].position - atoms[1].position))
    y.append(atoms.info['energy'])
r = np.array(r)
y = np.array(y)

popt, pcov = curve_fit(quadratic, r, y, p0=p0, bounds= bounds, maxfev=100000)
print(popt)
plt.scatter(r, y)
plt.plot(r, quadratic(r, *popt), color='red')
plt.show()